In [43]:
import numpy as np
import pandas as pd
import random 
import pickle
import json


In [44]:
import nltk

In [45]:
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout

In [46]:
!pip install tensorflow

In [47]:
from tensorflow.keras.optimizers import SGD

In [48]:
lemmatizer = WordNetLemmatizer()

In [87]:
intents = json.loads(open('intents.json').read())

In [88]:
words=[]
classes=[]
documents=[]
ignore_letters = ['?', '!', '.', ',']

In [89]:
for intent in intents['intents'] :
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
        

In [90]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dheer\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [91]:
words

['hi',
 'hello',
 'how',
 'are',
 'you',
 '?',
 'howdy',
 'what',
 'is',
 'your',
 'name',
 'what',
 'can',
 'i',
 'call',
 'you',
 'can',
 'you',
 'tell',
 'me',
 'your',
 'name',
 'may',
 'i',
 'know',
 'farmer',
 'plans',
 'farmer',
 'plans',
 'planning',
 'financing',
 'plans',
 'I',
 'am',
 'going',
 'to',
 'die',
 'I',
 'need',
 'some',
 'emotional',
 'support',
 'please',
 'help',
 'me',
 'how',
 'can',
 'i',
 'navigate',
 'in',
 'you',
 '?',
 'how',
 'can',
 'i',
 'get',
 'full',
 'use',
 'from',
 'you',
 '?',
 'what',
 'can',
 'you',
 'do',
 '?',
 'thank',
 'you',
 'see',
 'you',
 '!',
 '!',
 'bye',
 'good',
 'bye']

In [92]:
documents

[(['hi'], 'greetings'),
 (['hello'], 'greetings'),
 (['how', 'are', 'you', '?'], 'greetings'),
 (['howdy'], 'greetings'),
 (['what', 'is', 'your', 'name'], 'name'),
 (['what', 'can', 'i', 'call', 'you'], 'name'),
 (['can', 'you', 'tell', 'me', 'your', 'name'], 'name'),
 (['may', 'i', 'know', 'farmer', 'plans'], 'plans'),
 (['farmer', 'plans'], 'plans'),
 (['planning'], 'plans'),
 (['financing', 'plans'], 'plans'),
 (['I', 'am', 'going', 'to', 'die'], 'emotional support'),
 (['I', 'need', 'some', 'emotional', 'support'], 'emotional support'),
 (['please', 'help', 'me'], 'emotional support'),
 (['how', 'can', 'i', 'navigate', 'in', 'you', '?'], 'use'),
 (['how', 'can', 'i', 'get', 'full', 'use', 'from', 'you', '?'], 'use'),
 (['what', 'can', 'you', 'do', '?'], 'use'),
 (['thank', 'you', 'see', 'you', '!', '!'], 'end'),
 (['bye'], 'end'),
 (['good', 'bye'], 'end')]

In [93]:
words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]

In [94]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\dheer\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [95]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\dheer\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [96]:
words = sorted(set(words))

In [97]:
words

['I',
 'am',
 'are',
 'bye',
 'call',
 'can',
 'die',
 'do',
 'emotional',
 'farmer',
 'financing',
 'from',
 'full',
 'get',
 'going',
 'good',
 'hello',
 'help',
 'hi',
 'how',
 'howdy',
 'i',
 'in',
 'is',
 'know',
 'may',
 'me',
 'name',
 'navigate',
 'need',
 'plan',
 'planning',
 'please',
 'see',
 'some',
 'support',
 'tell',
 'thank',
 'to',
 'use',
 'what',
 'you',
 'your']

In [98]:
classes = sorted(set(classes))

In [99]:
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

In [100]:
training =[]
output_empty =[0]*  len(classes)

In [101]:
for document in documents:
    bag =[]
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
    output_row = list(output_empty)
    output_row[classes.index(document[1])]=1
    training.append([bag, output_row])

In [102]:
random.shuffle(training)
training=np.array(training)

C:\Users\dheer\AppData\Local\Temp\ipykernel_4992\4111924452.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training=np.array(training)


In [103]:
train_x = list(training[:, 0])
train_y = list(training[:,1])

In [104]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),) ,activation= 'relu'))

In [105]:
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='relu'))

In [106]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9)

In [107]:
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


In [108]:
hist = model.fit(np.array(train_x), np.array(train_y),epochs=200, batch_size=32, verbose=1)

Epoch 1/200
1/1 [==============================] - 1s 958ms/step - loss: 8.9691 - accuracy: 0.1500
Epoch 2/200
1/1 [==============================] - 0s 8ms/step - loss: 8.2874 - accuracy: 0.1500
Epoch 3/200
1/1 [==============================] - 0s 8ms/step - loss: 10.3337 - accuracy: 0.1500
Epoch 4/200
1/1 [==============================] - 0s 15ms/step - loss: 9.1910 - accuracy: 0.1500
Epoch 5/200
1/1 [==============================] - 0s 4ms/step - loss: 10.4746 - accuracy: 0.1500
Epoch 6/200
1/1 [==============================] - 0s 3ms/step - loss: 7.2688 - accuracy: 0.1500
Epoch 7/200
1/1 [==============================] - 0s 0s/step - loss: 8.9960 - accuracy: 0.1500
Epoch 8/200
1/1 [==============================] - 0s 11ms/step - loss: 6.3714 - accuracy: 0.1500
Epoch 9/200
1/1 [==============================] - 0s 8ms/step - loss: 5.3354 - accuracy: 0.1500
Epoch 10/200
1/1 [==============================] - 0s 9ms/step - loss: 5.6493 - accuracy: 0.1500
Epoch 11/200
1/1 [======

In [109]:
model.save('chatbot_model.h5', hist)

In [110]:
#using the model

In [111]:
from tensorflow.keras.models import load_model

In [112]:
words = pickle.load(open('words.pkl', 'rb'))

In [113]:
classes =  pickle.load(open('classes.pkl', 'rb'))

In [114]:
model = load_model("chatbot_model.h5")

In [115]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words=[lemmatizer.lemmatize(word) for word in sentence_words]
    return sentence_words

In [116]:
def bag_of_words(sentence):
    sentence_words = clean_up_sentence(sentence)
    bag =[0]*len(words)
    
    for w in sentence_words:
        for i, word in enumerate(words):
            if word==w:
                bag[i]=1
    return np.array(bag)

In [117]:
def predict_class(sentence):
    bow = bag_of_words(sentence)
    res= model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD=0.25
    results =[[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    
    results.sort(key= lambda x:x[1], reverse=True)
    
    return_list =[]
    
    for r in results:
        return_list.append({'intent':classes[r[0]], 'probability': str(r[1])})
    return return_list
    

In [118]:
def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    list_of_intents=intents_json['intents']
    for i in list_of_intents:
        if i['tag']==tag:
            result= random.choice(i['responses'])
            break
    return result

In [119]:
message = "can i use service"
ints = predict_class(message)
get_response(ints, intents)

1/1 [==============================] - 0s 120ms/step


'thanks for using our service hope we see you again'

In [120]:
ans = get_response(ints, intents)

In [121]:
type(ans)

str

In [122]:
ans

'check out our services might help you in some way'